In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [2]:
IMAGE_HEIGHT = 2160
IMAGE_WIDTH = 3840
COLORS = 3

In [3]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
encoded = tokenizer.encode("Hello world, how are you? Is this a lot?", return_tensors="pt")

In [5]:
encoded

tensor([[    1, 22557,  1526, 28725,   910,   460,   368, 28804,  1691,   456,
           264,  2055, 28804]])

In [21]:
total_image_size = IMAGE_HEIGHT * IMAGE_WIDTH * COLORS
difference = total_image_size - len(encoded.squeeze())
if difference < 0: raise Exception("NOT IMPLEMENTED")

left_half = (difference) // 2
right_half = left_half
if left_half != (difference - left_half):
  print(f"difference: {difference-left_half}, half: {left_half}")
  right_half = difference - left_half

expand_nums_times = max(IMAGE_HEIGHT, IMAGE_WIDTH)
normalized_encoding = encoded.to(torch.float64) / (255 * expand_nums_times)
repeated = normalized_encoding.repeat(int(expand_nums_times / encoded.shape[0])) if expand_nums_times == IMAGE_HEIGHT else normalized_encoding.repeat(1, int(expand_nums_times / encoded.shape[1]))
padded = F.pad(normalized_encoding, [left_half, right_half])
reshaped_padded = padded.reshape((COLORS, IMAGE_HEIGHT, IMAGE_WIDTH))

difference: 12441594, half: 12441593


In [22]:
repeated.shape

torch.Size([1, 3835])

In [7]:
(normalized_encoding * (255 * total_image_size)).to(torch.int32)

tensor([[     6480, 146169360,   9888480, 186138000,   5896800,   2980800,
           2384640, 186649920,  10957680,   2954880,   1710720,  13316400,
         186649920]], dtype=torch.int32)

In [10]:
tokenizer.decode((normalized_encoding * (255 * total_image_size)).to(torch.int32).squeeze().tolist())

'radio'

# Image Concatenation

In [11]:
transform = transforms.Compose([transforms.PILToTensor()])

image = transform(Image.open("BackgroundPicture1.jpg"))

In [12]:
image.shape

torch.Size([3, 2160, 3840])

In [13]:
(image + reshaped_padded).numpy().astype(np.uint8).shape

(3, 2160, 3840)

In [14]:
# result = Image.fromarray((image + reshaped_padded).numpy().astype(np.uint8).reshape(IMAGE_WIDTH, IMAGE_HEIGHT, COLORS))
# reshaped_padded[reshaped_padded != 0] = 200
result = transforms.ToPILImage()((image + reshaped_padded.to(torch.uint8)))
result.save("resultTest.jpg")

In [15]:
# TODO: Converting to uint8 causes all values to zero out. How do we avoid this?
reshaped_padded[reshaped_padded != 0].to(torch.uint8)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)

In [16]:
test = (transforms.PILToTensor()(result) - image)
test[test != 0]

tensor([], dtype=torch.uint8)